In [1]:
from sklearn.grid_search import GridSearchCV
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.model_selection import KFold, train_test_split, TimeSeriesSplit
from sklearn import linear_model
import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import log_loss, mean_squared_error as mse, r2_score 
from sklearn.metrics.scorer import make_scorer

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
def CVScore(model, X_train, y_train, n_splits=5, is_TimeSeries=False, seed=2017, my_score=mse):
    cv_scores = []
    X_train=np.array(X_train)
    y_train=np.array(y_train)
    if not is_TimeSeries:
        kf=KFold(n_splits=n_splits, shuffle=True, random_state=17)
    else:
        kf=TimeSeriesSplit(n_splits=n_splits)
    for train_idx, test_idx in kf.split(np.arange(len(X_train))):
        X_CVtrain = X_train[train_idx]
        y_CVtrain = y_train[train_idx]
        X_CVholdout = X_train[test_idx]
        y_CVholdout = y_train[test_idx]
        model.fit(X_CVtrain, y_CVtrain)
        if my_score==log_loss:
            pred=model.predict_proba(X_CVholdout)
        else:
            pred = model.predict(X_CVholdout)[:]
        cv_scores.append(my_score(y_CVholdout, pred))
    return np.mean(cv_scores)

In [3]:
n=1000
np.random.seed(17)
X=pd.DataFrame(np.random.randn(n,1))
y=X.iloc[:,0]+.2*pd.Series(np.random.randn(n))
X_train,y_train=X.iloc[:n/2], y.iloc[:n/2]
X_test, y_test=X.iloc[n/2:], y.iloc[n/2:]

In [5]:
CVScore(linear_model.LinearRegression(fit_intercept=False), X_train, y_train, my_score=mse)

0.043842510288573847